# PLI v0 Demo (Modeling Notebook)

이 노트북은 **가중치 곱셈 기반 PLI(v0)** 데모입니다.  
- 환경적 요인: 날씨(기온)  
- 상황적 요인: 연패, 실책, 대타/대주자, 고의4구  
- 개인적 요인: 컨디션(최근5경기/시즌), 연차  
- **상황 레벨**: 이닝/아웃/주자/점수차 기반 (0~11점 → L1~L4)

> 목표: **WPA × (모든 가중치의 곱)** → PLI 산출  
> WE/WPA, WAR, 파크팩터는 후속 버전에서 붙일 수 있도록 자리만 남겨둠.


In [1]:
# Imports
from dataclasses import dataclass
from typing import Optional, Tuple, Dict, Any
import math
import pandas as pd
import numpy as np
from IPython.display import display

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)


## 날씨 불러오기

In [ ]:
import requests
import json

# URL 문자열
url = 'https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-dfs_shrt_grd?tmfc=2025081818&tmef=2025081824&vars=TMP&authKey=mhFQBEknT9eRUARJJ3_XYA&reg=11H20301'

# GET 요청
response = requests.get(url)
response.text

'#START7777\n#--------------------------------------------------------------------------------------------------\n#  단기예보 육상 조회 [입력인수형태][예] ?reg=&tmfc1=2013121018&tmfc2=2013121106&disp=0&help=1\n#--------------------------------------------------------------------------------------------------\n#  1. REG_ID   : 예보구역코드\n#  2. TM_FC    : 발표시각(년월일시분,KST)\n#  3. TM_EF    : 발효시각(년월일시분,KST)\n#  4. MOD      : 구간 (A01(24시간),A02(12시간))\n#  5. NE       : 발효번호\n#  6. STN      : 발표관서\n#  7. C        : 발표코드\n#  8. MAN_ID   : 예보관ID\n#  9. MAN_FC   : 예보관명\n# 10. W1       : 풍향1(16방위)\n# 11. T        : 풍향경향(1:-, 2:후)\n# 12. W2       : 풍향2(16방위)\n# 13. TA       : 기온\n# 14. ST       : 강수확률(%)\n# 15. SKY      : 하늘상태코드 (DB01(맑음),DB02(구름조금),DB03(구름많음),DB04(흐림))\n# 16. PREP     : 강수유무코드 (0(없음),1(비),2(비/눈),3(눈),4(눈/비(~\'19.6.4.),소나기(\'19.6.4~)))\n# 17. WF       : 예보\n#--------------------------------------------------------------------------------------------------\n# REG_ID TM_FC        TM_EF        MOD NE S

In [30]:
# pli_api/weather/kma_short_land_parser.py
import re
from datetime import datetime, timedelta, timezone

KST = timezone(timedelta(hours=9))

DATA_RE = re.compile(r"""
    ^\s*
    (?P<REG_ID>\S+)\s+
    (?P<TM_FC>\d{12})\s+
    (?P<TM_EF>\d{12})\s+
    (?P<MOD>\S+)\s+
    (?P<NE>\S+)\s+
    (?P<STN>\S+)\s+
    (?P<C>\S+)\s+
    (?P<MAN_ID>\S+)\s+
    (?P<MAN_FC>\S+)\s+
    (?P<W1>\S+)\s+
    (?P<T>\S+)\s+
    (?P<W2>\S+)\s+
    (?P<TA>-?\d+)\s+
    (?P<ST>\d+)\s+
    (?P<SKY>\S+)\s+
    (?P<PREP>\S+)\s+
    (?P<WF>.+?)\s*$
""", re.VERBOSE)

def parse_short_land_text(table_text: str):
    rows = []
    in_data = False
    for raw in table_text.splitlines():
        line = raw.strip()
        if not line:
            continue
        if line.startswith("#7777END"):
            break
        if line.startswith("# REG_ID"):
            in_data = True
            continue
        if not in_data or line.startswith("#"):
            continue

        m = DATA_RE.match(line)
        if not m:
            continue

        d = m.groupdict()
        d["TM_FC_dt"] = datetime.strptime(d["TM_FC"], "%Y%m%d%H%M").replace(tzinfo=KST).isoformat()
        d["TM_EF_dt"] = datetime.strptime(d["TM_EF"], "%Y%m%d%H%M").replace(tzinfo=KST).isoformat()
        wf = d["WF"].strip()
        if wf.startswith('"') and wf.endswith('"'):
            wf = wf[1:-1]
        d["WF"] = wf
        d["TA_c"] = None if d["TA"] in ("", "-99") else float(d["TA"])
        d["ST_pct"] = int(d["ST"])
        rows.append(d)
    return rows

In [31]:
parse_short_land_text(response.text)

[{'REG_ID': '11H20301',
  'TM_FC': '202508181100',
  'TM_EF': '202508181200',
  'MOD': 'A02',
  'NE': '0',
  'STN': '159',
  'C': '2',
  'MAN_ID': 'wth*********',
  'MAN_FC': '김중락',
  'W1': 'SW',
  'T': '1',
  'W2': 'W',
  'TA': '33',
  'ST': '0',
  'SKY': 'DB01',
  'PREP': '0',
  'WF': '맑음',
  'TM_FC_dt': '2025-08-18T11:00:00+09:00',
  'TM_EF_dt': '2025-08-18T12:00:00+09:00',
  'TA_c': 33.0,
  'ST_pct': 0},
 {'REG_ID': '11H20301',
  'TM_FC': '202508181100',
  'TM_EF': '202508190000',
  'MOD': 'A02',
  'NE': '1',
  'STN': '159',
  'C': '2',
  'MAN_ID': 'wth*********',
  'MAN_FC': '김중락',
  'W1': 'SW',
  'T': '1',
  'W2': 'W',
  'TA': '25',
  'ST': '20',
  'SKY': 'DB03',
  'PREP': '0',
  'WF': '구름많음',
  'TM_FC_dt': '2025-08-18T11:00:00+09:00',
  'TM_EF_dt': '2025-08-19T00:00:00+09:00',
  'TA_c': 25.0,
  'ST_pct': 20},
 {'REG_ID': '11H20301',
  'TM_FC': '202508181100',
  'TM_EF': '202508191200',
  'MOD': 'A02',
  'NE': '2',
  'STN': '159',
  'C': '2',
  'MAN_ID': 'wth*********',
  'MAN_FC

## 가중치 함수 (v0)

In [2]:
def weather_weight(temp_c: Optional[float], base: float = 22.0, alpha: float = 0.02) -> float:
    """기온 가중치. 기준 이하일 때 1.0 고정."""
    if temp_c is None or temp_c <= base:
        return 1.0
    return 1.0 + (temp_c - base) * alpha

def streak_weight(loss_streak: Optional[int]) -> float:
    if loss_streak is None or loss_streak <= 2: return 1.0
    if loss_streak <= 4: return 0.95
    if loss_streak <= 6: return 0.90
    return 0.85

def condition_weight(recent_ba: Optional[float], season_ba: Optional[float], beta: float = 0.5) -> float:
    if not season_ba or season_ba == 0 or not recent_ba:
        return 1.0
    idx = recent_ba / season_ba
    return 1.0 + (idx - 1.0) * beta

def experience_weight(years_pro: Optional[int]) -> float:
    if years_pro is None: return 1.0
    y = int(years_pro)
    if y <= 1: return 0.9
    if y >= 7: return 1.1
    return 1.0

def pinch_weight(flag: bool) -> float:
    return 1.8 if flag else 1.0

def ibb_focus_weight(flag: bool) -> float:
    return 2.0 if flag else 1.0

def error_momentum_weight(flag: bool, next_batter: bool=False) -> float:
    if not flag: return 1.0
    return 1.1 if next_batter else 1.3

def combine_weights(*ws: float) -> float:
    v = 1.0
    for w in ws:
        if w and w > 0:
            v *= float(w)
    return v


## 상황 레벨 (0~11점 → L1~L4)

In [3]:
def calculate_situation_level(inning: int, outs: int, runners_code: str, score_diff: int):
    total = 0
    # 이닝
    if inning >= 9: total += 3
    elif inning == 8: total += 2
    elif inning == 7: total += 1
    # 아웃
    if outs == 0: total += 2
    elif outs == 1: total += 1
    # 주자
    runner_scores = {
        '000':0,'100':1,'010':2,'110':2,'001':3,'101':3,'011':3,'111':3
    }
    total += runner_scores.get(runners_code, 0)
    # 점수차(절대값)
    d = abs(score_diff)
    if d == 0: total += 3
    elif d == 1: total += 2
    elif d <= 3: total += 1
    # 레벨
    if total >= 9: return 4, "최대 위기/방화", total
    if total >= 6: return 3, "중요한 승부처", total
    if total >= 3: return 2, "관리 필요", total
    return 1, "일상적 교체", total


## PLI 계산기

In [4]:
def compute_pli_row(row: pd.Series) -> Dict[str, Any]:
    # 상황 레벨
    level, desc, score = calculate_situation_level(
        int(row['inning']), int(row['outs']), str(row['runners_code']), int(row['score_diff'])
    )
    # 가중치
    w_env = weather_weight(row.get('temp_c'))
    w_personal = combine_weights(
        condition_weight(row.get('recent_ba'), row.get('season_ba')),
        experience_weight(row.get('years_pro'))
    )
    w_situ = combine_weights(
        streak_weight(row.get('loss_streak')),
        pinch_weight(bool(row.get('pinch_event', False))),
        ibb_focus_weight(bool(row.get('intentional_walk', False))),
        error_momentum_weight(bool(row.get('error_happened', False)), bool(row.get('next_batter_after_error', False)))
    )
    w_total = combine_weights(w_env, w_personal, w_situ)
    pli = float(row['base_wpa']) * w_total
    return {
        "pli": round(pli, 4),
        "w_env": round(w_env, 4),
        "w_personal": round(w_personal, 4),
        "w_situ": round(w_situ, 4),
        "w_total": round(w_total, 4),
        "level": level, "level_desc": desc, "level_score": score
    }


## 데모 시나리오

In [5]:
demo = pd.DataFrame([
    # 예시 1: 8회, 1아웃, 1/3루, 1점차, 무더위, 대타, 실책 직후
    {"name":"hot_pinch_error",
     "base_wpa":0.18, "inning":8, "outs":1, "runners_code":"101", "score_diff":1,
     "temp_c":29, "loss_streak":5, "recent_ba":0.45, "season_ba":0.28, "years_pro":8,
     "pinch_event":True, "intentional_walk":False, "error_happened":True, "next_batter_after_error":False},

    # 예시 2: 3회, 2아웃, 주자없음, 5점차 패배중, 선선함, 휴지기, 신인
    {"name":"low_leverage_rookie",
     "base_wpa":0.05, "inning":3, "outs":2, "runners_code":"000", "score_diff":-5,
     "temp_c":19, "loss_streak":1, "recent_ba":0.22, "season_ba":0.3, "years_pro":1,
     "pinch_event":False, "intentional_walk":False, "error_happened":False},

    # 예시 3: 동점 9회, 0아웃, 만루, IBB 직후, 베테랑, 컨디션 호조
    {"name":"tie_ninth_bases_loaded_ibb",
     "base_wpa":0.25, "inning":9, "outs":0, "runners_code":"111", "score_diff":0,
     "temp_c":24, "loss_streak":3, "recent_ba":0.38, "season_ba":0.29, "years_pro":10,
     "pinch_event":False, "intentional_walk":True, "error_happened":False},
])
demo


,name,base_wpa,inning,outs,runners_code,score_diff,temp_c,loss_streak,recent_ba,season_ba,years_pro,pinch_event,intentional_walk,error_happened,next_batter_after_error
0,hot_pinch_error,0.18,8,1,101,1,29,5,0.45,0.28,8,True,False,True,False
1,low_leverage_rookie,0.05,3,2,000,-5,19,1,0.22,0.30,1,False,False,False,NaN
2,tie_ninth_bases_loaded_ibb,0.25,9,0,111,0,24,3,0.38,0.29,10,False,True,False,NaN


## 계산 실행

In [6]:
out = demo.apply(compute_pli_row, axis=1, result_type='expand')
result = pd.concat([demo[['name','base_wpa','inning','outs','runners_code','score_diff']],
                    out[['w_env','w_personal','w_situ','w_total','level','level_desc','pli']]], axis=1)
display(result)

,name,base_wpa,inning,outs,runners_code,score_diff,w_env,w_personal,w_situ,w_total,level,level_desc,pli
0,hot_pinch_error,0.18,8,1,101,1,1.14,1.4339,2.106,3.4426,3,중요한 승부처,0.6197
1,low_leverage_rookie,0.05,3,2,000,-5,1.00,0.7800,1.000,0.7800,1,일상적 교체,0.0390
2,tie_ninth_bases_loaded_ibb,0.25,9,0,111,0,1.04,1.2707,1.900,2.5109,4,최대 위기/방화,0.6277


In [32]:
# adapters/pli_adapter.py
import re
from typing import Dict, Any, List, Optional

KW_PINCH = ("대타", "대주자")
KW_IBB = ("고의4구", "자동고의")
KW_ERROR = ("실책",)

def runners_code_from_onbase(on_base: Dict[str, str]) -> str:
    """
    on_base 예: {"base1":"0","base2":"9","base3":"0"}
    '0'이 아니면 1(점유)로 본다.
    반환: 'XYZ' (1루-2루-3루) → '101' 등
    """
    def occ(v: Optional[str]) -> str:
        return "1" if (v is not None and str(v) != "0") else "0"
    b1 = occ(on_base.get("base1"))
    b2 = occ(on_base.get("base2"))
    b3 = occ(on_base.get("base3"))
    return f"{b1}{b2}{b3}"

def parse_score_pair(score: str, fmt: str = "home:away") -> (int, int):
    """score '4:9' → (home, away). fmt이 'away:home'이면 반대로 해석."""
    a, b = score.split(":")
    a, b = int(a), int(b)
    if fmt == "home:away":
        return a, b
    elif fmt == "away:home":
        return b, a
    else:
        raise ValueError("Unknown score format")

def batting_team_for_half(half: str, home_team: str, away_team: str) -> str:
    return away_team if half == "top" else home_team

def score_diff_for_batting(score: str, half: str, home_team: str, away_team: str, score_fmt: str = "home:away") -> int:
    """
    타자(공격) 팀 기준 점수차 = (타자 팀 득점 - 상대 득점)
    """
    home, away = parse_score_pair(score, fmt=score_fmt)
    batting_team = batting_team_for_half(half, home_team, away_team)
    if batting_team == home_team:
        return home - away
    else:
        return away - home

def text_has_any(s: Optional[str], keywords: tuple) -> bool:
    if not s:
        return False
    return any(kw in s for kw in keywords)

def detect_triggers(atbat: Dict[str, Any]) -> Dict[str, bool]:
    pinch = False
    ibb = False
    error = False

    # 1) 대타/대주자
    if atbat.get("original_batter") and atbat.get("actual_batter"):
        if atbat["original_batter"] != atbat["actual_batter"]:
            pinch = True
    # 텍스트 키워드
    if text_has_any(atbat.get("full_result"), KW_PINCH) or text_has_any(atbat.get("main_result"), KW_PINCH):
        pinch = True
    # pitch_sequence 중 event 텍스트도 훑기
    for ps in atbat.get("pitch_sequence", []):
        ev = ps.get("event")
        if isinstance(ev, str):
            if text_has_any(ev, KW_PINCH):
                pinch = True
        elif isinstance(ev, list):
            if any(text_has_any(x, KW_PINCH) for x in ev if isinstance(x, str)):
                pinch = True

    # 2) 고의4구
    if text_has_any(atbat.get("full_result"), KW_IBB) or text_has_any(atbat.get("main_result"), KW_IBB):
        ibb = True

    # 3) 실책
    if text_has_any(atbat.get("full_result"), KW_ERROR) or text_has_any(atbat.get("main_result"), KW_ERROR):
        error = True

    return {
        "pinch_event": pinch,
        "intentional_walk": ibb,
        "error_happened": error,
    }

def atbat_to_pli_payload(
    atbat: Dict[str, Any],
    context: Dict[str, Any],
    *,
    temp_c: Optional[float] = None,
    base_wpa: Optional[float] = None,
    loss_streak: Optional[int] = None,
    recent_ba: Optional[float] = None,
    season_ba: Optional[float] = None,
    years_pro: Optional[int] = None,
    score_format: str = "home:away",
) -> Dict[str, Any]:
    """
    한 타석 atbat → PLI 엔진 요청 바디로 변환
    context 예:
    {
      "home_team": "NC",
      "away_team": "HH"
    }
    """
    inning = int(atbat["inning"])
    outs = int(atbat["out"])
    half = atbat.get("half", "top")
    runners_code = runners_code_from_onbase(atbat.get("on_base", {}))
    score_str = atbat.get("score", "0:0")

    score_diff = score_diff_for_batting(
        score_str, half, context["home_team"], context["away_team"], score_fmt=score_format
    )

    triggers = detect_triggers(atbat)
    # NOTE: base_wpa가 없으면 임시 디폴트 (원하면 0.1~0.2 사이 전략적으로)
    bwpa = float(base_wpa if base_wpa is not None else 0.12)

    payload = {
        "base_wpa": bwpa,
        "inning": inning,
        "outs": outs,
        "runners_code": runners_code,
        "score_diff": score_diff,
        "temp_c": temp_c,
        # 팀/선수 요인은 optional — 값 없으면 가중치 1.0 처리
        "loss_streak": loss_streak,
        "recent_ba": recent_ba,
        "season_ba": season_ba,
        "years_pro": years_pro,
        # 트리거
        "pinch_event": triggers["pinch_event"],
        "intentional_walk": triggers["intentional_walk"],
        "error_happened": triggers["error_happened"],
        # 실책 다음 타자 보정은 상위에서 컨텍스트로 판단하여 True/False 넘겨도 됨
        "next_batter_after_error": False,
    }
    return payload

def inning_pack_to_pli_requests(feed: Dict[str, Any], *, temp_c: Optional[float] = None, score_format: str = "home:away") -> List[Dict[str, Any]]:
    """
    질문에 준 JSON(한 이닝의 top/bot 묶음)을 받아,
    각 at-bat을 PLI 요청 리스트로 변환
    """
    data = feed["data"]
    ctx = {
        "home_team": data["defense_positions"]["home_team"],
        "away_team": data["defense_positions"]["away_team"],
    }
    requests: List[Dict[str, Any]] = []

    def _absorb(side: str):
        side_block = data.get(side)
        if not side_block:
            return
        for ab in side_block.get("atbats", []):
            req = atbat_to_pli_payload(
                ab, ctx, temp_c=temp_c, score_format=score_format
            )
            requests.append(req)

    _absorb("top")
    _absorb("bot")
    return requests

def inning_pack_to_pli_scores(
    feed: dict, *, temp_c: float | None = None, score_format: str = "home:away",
    return_detail: bool = False,
):
    """
    return_detail=False: [0.1372, 0.0921, ...]  # 타석 순서대로 PLI 숫자만
    return_detail=True : [{"pli":..., "weights":..., "situation":..., "input":...}, ...]
    """
    reqs = inning_pack_to_pli_requests(feed, temp_c=temp_c, score_format=score_format)
    results = []
    for r in reqs:
        level, desc, total_score = calculate_situation_level(
            r["inning"], r["outs"], r["runners_code"], r["score_diff"]
        )
        w_env = weather_weight(r.get("temp_c"))
        w_personal = combine_weights(
            condition_weight(r.get("recent_ba"), r.get("season_ba")),
            experience_weight(r.get("years_pro")),
        )
        w_situ = combine_weights(
            streak_weight(r.get("loss_streak")),
            pinch_weight(bool(r.get("pinch_event"))),
            ibb_focus_weight(bool(r.get("intentional_walk"))),
            error_momentum_weight(bool(r.get("error_happened")), bool(r.get("next_batter_after_error")))
        )
        w_total = combine_weights(w_env, w_personal, w_situ)
        pli = float(r["base_wpa"]) * w_total
        pli_val = round(pli, 4)

        if return_detail:
            results.append({
                "input": r,
                "pli": pli_val,
                "weights": {
                    "environment": round(w_env, 4),
                    "personal": round(w_personal, 4),
                    "situational": round(w_situ, 4),
                    "combined": round(w_total, 4),
                },
                "situation": {"level": level, "description": desc, "score": total_score},
            })
        else:
            results.append(pli_val)
    return results

## 더미데이터

In [34]:
import json

raw_data = json.loads("""
{
    "status": "OK_REALTIME",
    "message": "9회 이닝 정보 (실시간)",
    "data": {
        "top": {
            "game": "20250815HHNC02025",
            "inning_number": 9,
            "atbats": [
                {
                    "inning": 9,
                    "half": "top",
                    "pitcher": "52992",
                    "bat_order": 8,
                    "original_batter": null,
                    "actual_batter": "78288",
                    "out": "0",
                    "score": "6:2",
                    "on_base": {
                        "base1": "8",
                        "base2": "0",
                        "base3": "0"
                    },
                    "appearance_number": 1,
                    "strike_zone": [
                        3.241,
                        1.572,
                        0.75,
                        -0.75
                    ],
                    "full_result": "9회초 8번타순 10구 체크스윙 → 노 스윙 : 한화 최재훈",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.24329661094573862,
                                    2.5761973494323676
                                ]
                            ],
                            "speed": "143",
                            "count": "0-1",
                            "pitch_result": "스트라이크",
                            "event": null
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.8336798135282728,
                                    1.5261444742144614
                                ]
                            ],
                            "speed": "145",
                            "count": "0-2",
                            "pitch_result": "스트라이크",
                            "event": null
                        },
                        {
                            "pitch_num": 3,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    1.4529255546021904,
                                    2.559007338995863
                                ]
                            ],
                            "speed": "143",
                            "count": "1-2",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 4,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    0.702147611301194,
                                    1.8233275413919001
                                ]
                            ],
                            "speed": "131",
                            "count": "1-2",
                            "pitch_result": "파울",
                            "event": null
                        },
                        {
                            "pitch_num": 5,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    1.4210670945795632,
                                    3.7663946955403604
                                ]
                            ],
                            "speed": "145",
                            "count": "2-2",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 6,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    -0.8095409322745795,
                                    3.9387431703955658
                                ]
                            ],
                            "speed": "129",
                            "count": "3-2",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 7,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    -0.02783243960664472,
                                    1.8781848788812106
                                ]
                            ],
                            "speed": "133",
                            "count": "3-2",
                            "pitch_result": "파울",
                            "event": null
                        },
                        {
                            "pitch_num": 8,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    0.05236525554944599,
                                    1.964196311108453
                                ]
                            ],
                            "speed": "134",
                            "count": "3-2",
                            "pitch_result": "파울",
                            "event": null
                        },
                        {
                            "pitch_num": 9,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    0.3406670849886363,
                                    2.72577803018826
                                ]
                            ],
                            "speed": "133",
                            "count": "3-2",
                            "pitch_result": "파울",
                            "event": null
                        },
                        {
                            "pitch_num": 10,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    -0.11430030395912949,
                                    3.670121478893454
                                ]
                            ],
                            "speed": "145",
                            "count": "4-2",
                            "pitch_result": "볼",
                            "event": null
                        }
                    ],
                    "main_result": "볼넷"
                },
                {
                    "inning": 9,
                    "half": "top",
                    "pitcher": "52992",
                    "bat_order": 9,
                    "original_batter": null,
                    "actual_batter": "64006",
                    "out": "0",
                    "score": "6:2",
                    "on_base": {
                        "base1": "9",
                        "base2": "8",
                        "base3": "0"
                    },
                    "appearance_number": 1,
                    "strike_zone": [
                        3.34,
                        1.62,
                        0.75,
                        -0.75
                    ],
                    "full_result": "1루주자 하주석 : 2루까지 진루",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.08564469843110101,
                                    2.9387716888482194
                                ]
                            ],
                            "speed": "145",
                            "count": "0-1",
                            "pitch_result": "스트라이크",
                            "event": "1루주자 최재훈 : 대주자 하주석 (으)로 교체|투수 투수판 이탈"
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    -1.8054607871604451,
                                    3.327352459336841
                                ]
                            ],
                            "speed": "131",
                            "count": "1-1",
                            "pitch_result": "볼",
                            "event": null
                        }
                    ],
                    "main_result": "몸에 맞는 볼"
                },
                {
                    "inning": 9,
                    "half": "top",
                    "pitcher": "52992",
                    "bat_order": 1,
                    "original_batter": null,
                    "actual_batter": "52764",
                    "out": "0",
                    "score": "6:2",
                    "on_base": {
                        "base1": "9",
                        "base2": "8",
                        "base3": "0"
                    },
                    "appearance_number": 1,
                    "strike_zone": null,
                    "full_result": "1번타자 손아섭 : 대타 허인서 (으)로 교체",
                    "pitch_sequence": []
                },
                {
                    "inning": 9,
                    "half": "top",
                    "pitcher": "52992",
                    "bat_order": 1,
                    "original_batter": null,
                    "actual_batter": "52764",
                    "out": "2",
                    "score": "6:2",
                    "on_base": {
                        "base1": "0",
                        "base2": "0",
                        "base3": "8"
                    },
                    "appearance_number": 2,
                    "strike_zone": [
                        3.365,
                        1.632,
                        0.75,
                        -0.75
                    ],
                    "full_result": "대타 허인서|허인서 : 2루수 병살타 아웃 (2루수->유격수->1루수 송구아웃)|1루주자 심우준 : 포스아웃 (2루수->유격수 2루 터치아웃)|2루주자 하주석 : 3루까지 진루",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    0.047695971814526644,
                                    1.4932743878739583
                                ]
                            ],
                            "speed": "132",
                            "count": "0-1",
                            "pitch_result": "헛스윙",
                            "event": null
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    0.6223723129303991,
                                    1.6517441885892956
                                ]
                            ],
                            "speed": "131",
                            "count": "0-2",
                            "pitch_result": "헛스윙",
                            "event": null
                        },
                        {
                            "pitch_num": 3,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    0.2921863157416469,
                                    0.6272210678100341
                                ]
                            ],
                            "speed": "132",
                            "count": "0-2",
                            "pitch_result": "타격",
                            "event": null
                        }
                    ]
                },
                {
                    "inning": 9,
                    "half": "top",
                    "pitcher": "66920",
                    "bat_order": 2,
                    "original_batter": null,
                    "actual_batter": "55703",
                    "out": "2",
                    "score": "6:2",
                    "on_base": {
                        "base1": "2",
                        "base2": "0",
                        "base3": "8"
                    },
                    "appearance_number": 1,
                    "strike_zone": [
                        3.417,
                        1.657,
                        0.75,
                        -0.75
                    ],
                    "full_result": "리베라토 : 볼넷",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.11781421914619217,
                                    1.3863201510485417
                                ]
                            ],
                            "speed": "136",
                            "count": "1-0",
                            "pitch_result": "볼",
                            "event": "투수 이준혁 : 투수 최성영 (으)로 교체"
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    -0.964361435484117,
                                    3.2092061357393193
                                ]
                            ],
                            "speed": "123",
                            "count": "2-0",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 3,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    -0.24882700755065246,
                                    1.9338915108629728
                                ]
                            ],
                            "speed": "139",
                            "count": "2-1",
                            "pitch_result": "스트라이크",
                            "event": null
                        },
                        {
                            "pitch_num": 4,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    -1.608706566191269,
                                    0.5425127843235655
                                ]
                            ],
                            "speed": "125",
                            "count": "3-1",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 5,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    -0.8152578474425323,
                                    1.1853016646577343
                                ]
                            ],
                            "speed": "139",
                            "count": "4-1",
                            "pitch_result": "볼",
                            "event": null
                        }
                    ],
                    "main_result": "볼넷"
                },
                {
                    "inning": 9,
                    "half": "top",
                    "pitcher": "66920",
                    "bat_order": 3,
                    "original_batter": null,
                    "actual_batter": "53764",
                    "out": "2",
                    "score": "9:2",
                    "on_base": {
                        "base1": "0",
                        "base2": "0",
                        "base3": "0"
                    },
                    "appearance_number": 1,
                    "strike_zone": [
                        3.17,
                        1.537,
                        0.75,
                        -0.75
                    ],
                    "full_result": "9회초 3번타순 2구 체크스윙 → 노 스윙 : 한화 문현빈|문현빈 : 우익수 뒤 홈런 (홈런거리:120M)|1루주자 리베라토 : 홈인|3루주자 하주석 : 홈인",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.13216487701923674,
                                    2.7162858254605293
                                ]
                            ],
                            "speed": "141",
                            "count": "0-1",
                            "pitch_result": "파울",
                            "event": null
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.27346128928310254,
                                    3.467988919921652
                                ]
                            ],
                            "speed": "141",
                            "count": "1-1",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 3,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.339184922597328,
                                    2.288975429188571
                                ]
                            ],
                            "speed": "138",
                            "count": "1-1",
                            "pitch_result": "타격",
                            "event": null
                        }
                    ]
                },
                {
                    "inning": 9,
                    "half": "top",
                    "pitcher": "66920",
                    "bat_order": 4,
                    "original_batter": null,
                    "actual_batter": "69737",
                    "out": "2",
                    "score": "9:2",
                    "on_base": {
                        "base1": "4",
                        "base2": "0",
                        "base3": "0"
                    },
                    "appearance_number": 1,
                    "strike_zone": [
                        3.349,
                        1.624,
                        0.75,
                        -0.75
                    ],
                    "full_result": "노시환 : 좌익수 오른쪽 1루타",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.4912340118019476,
                                    1.7541685091493786
                                ]
                            ],
                            "speed": "141",
                            "count": "0-1",
                            "pitch_result": "스트라이크",
                            "event": null
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "포크",
                            "pitch_coordinate": [
                                [
                                    0.8009185282829967,
                                    2.2209603793411383
                                ]
                            ],
                            "speed": "124",
                            "count": "0-1",
                            "pitch_result": "타격",
                            "event": null
                        }
                    ],
                    "main_result": "좌익수 오른쪽 1루타"
                },
                {
                    "inning": 9,
                    "half": "top",
                    "pitcher": "66920",
                    "bat_order": 5,
                    "original_batter": null,
                    "actual_batter": "66704",
                    "out": "2",
                    "score": "9:2",
                    "on_base": {
                        "base1": "0",
                        "base2": "5",
                        "base3": "4"
                    },
                    "appearance_number": 1,
                    "strike_zone": [
                        3.259,
                        1.581,
                        0.75,
                        -0.75
                    ],
                    "full_result": "1루주자 노시환 : 3루까지 진루",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.43800618849095563,
                                    3.3611676205341436
                                ]
                            ],
                            "speed": "140",
                            "count": "1-0",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.07282903328279144,
                                    2.086065389188862
                                ]
                            ],
                            "speed": "138",
                            "count": "1-0",
                            "pitch_result": "타격",
                            "event": null
                        }
                    ],
                    "main_result": "좌익수 왼쪽 2루타"
                },
                {
                    "inning": 9,
                    "half": "top",
                    "pitcher": "66920",
                    "bat_order": 6,
                    "original_batter": null,
                    "actual_batter": "68700",
                    "out": "3",
                    "score": "9:2",
                    "on_base": {
                        "base1": "0",
                        "base2": "5",
                        "base3": "4"
                    },
                    "appearance_number": 1,
                    "strike_zone": [
                        3.254,
                        1.578,
                        0.75,
                        -0.75
                    ],
                    "full_result": "이원석 : 삼진 아웃",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    0.9125483887184911,
                                    5.052954664524453
                                ]
                            ],
                            "speed": "121",
                            "count": "1-0",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.4033114303068699,
                                    3.532009513756412
                                ]
                            ],
                            "speed": "141",
                            "count": "2-0",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 3,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    -0.5328251215007548,
                                    1.3475485544128083
                                ]
                            ],
                            "speed": "140",
                            "count": "3-0",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 4,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    -0.07957533617980395,
                                    2.879098109221778
                                ]
                            ],
                            "speed": "136",
                            "count": "3-1",
                            "pitch_result": "스트라이크",
                            "event": null
                        },
                        {
                            "pitch_num": 5,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.03264936231069082,
                                    1.9881459752132558
                                ]
                            ],
                            "speed": "139",
                            "count": "3-2",
                            "pitch_result": "파울",
                            "event": null
                        },
                        {
                            "pitch_num": 6,
                            "pitch_type": "포크",
                            "pitch_coordinate": [
                                [
                                    0.4200739295574205,
                                    1.8566700453094453
                                ]
                            ],
                            "speed": "122",
                            "count": "3-2",
                            "pitch_result": "파울",
                            "event": null
                        },
                        {
                            "pitch_num": 7,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    -0.7128730572276658,
                                    1.4477182132911104
                                ]
                            ],
                            "speed": "140",
                            "count": "3-3",
                            "pitch_result": "스트라이크",
                            "event": null
                        }
                    ],
                    "main_result": "삼진 아웃"
                }
            ]
        },
        "bot": {
            "game": "20250815HHNC02025",
            "inning_number": 9,
            "atbats": [
                {
                    "inning": 9,
                    "half": "bot",
                    "pitcher": "61666",
                    "bat_order": 6,
                    "original_batter": null,
                    "actual_batter": "69992",
                    "out": "0",
                    "score": "9:2",
                    "on_base": {
                        "base1": "0",
                        "base2": "0",
                        "base3": "0"
                    },
                    "appearance_number": 1,
                    "strike_zone": null,
                    "full_result": "대타 허인서 : 포수(으)로 수비위치 변경|6번타자 이우성 : 대타 최정원 (으)로 교체",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "event": [
                                "대주자 하주석 : 투수 한승혁 (으)로 교체"
                            ]
                        }
                    ]
                },
                {
                    "inning": 9,
                    "half": "bot",
                    "pitcher": "61666",
                    "bat_order": 6,
                    "original_batter": null,
                    "actual_batter": "69992",
                    "out": "0",
                    "score": "9:2",
                    "on_base": {
                        "base1": "6",
                        "base2": "0",
                        "base3": "0"
                    },
                    "appearance_number": 2,
                    "strike_zone": [
                        3.245,
                        1.574,
                        0.75,
                        -0.75
                    ],
                    "full_result": "대타 최정원|최정원 : 몸에 맞는 볼",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    -0.27497515960021657,
                                    3.0290136243513848
                                ]
                            ],
                            "speed": "133",
                            "count": "0-1",
                            "pitch_result": "스트라이크",
                            "event": null
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "포크",
                            "pitch_coordinate": [
                                [
                                    -0.5500841406573953,
                                    2.637500512695186
                                ]
                            ],
                            "speed": "133",
                            "count": "0-2",
                            "pitch_result": "파울",
                            "event": null
                        },
                        {
                            "pitch_num": 3,
                            "pitch_type": "포크",
                            "pitch_coordinate": [
                                [
                                    -1.682706606376398,
                                    4.230288681537843
                                ]
                            ],
                            "speed": "135",
                            "count": "1-2",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 4,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    -0.633582700382768,
                                    2.5280461324174577
                                ]
                            ],
                            "speed": "146",
                            "count": "1-2",
                            "pitch_result": "파울",
                            "event": null
                        },
                        {
                            "pitch_num": 5,
                            "pitch_type": "포크",
                            "pitch_coordinate": [
                                [
                                    1.0796329384896899,
                                    -0.8945528020958866
                                ]
                            ],
                            "speed": "133",
                            "count": "2-2",
                            "pitch_result": "볼",
                            "event": null
                        }
                    ]
                },
                {
                    "inning": 9,
                    "half": "bot",
                    "pitcher": "61666",
                    "bat_order": 7,
                    "original_batter": null,
                    "actual_batter": "69995",
                    "out": "0",
                    "score": "9:2",
                    "on_base": {
                        "base1": "6",
                        "base2": "0",
                        "base3": "0"
                    },
                    "appearance_number": 1,
                    "strike_zone": null,
                    "full_result": "7번타자 김휘집 : 대타 서호철 (으)로 교체",
                    "pitch_sequence": []
                },
                {
                    "inning": 9,
                    "half": "bot",
                    "pitcher": "61666",
                    "bat_order": 7,
                    "original_batter": null,
                    "actual_batter": "69995",
                    "out": "1",
                    "score": "9:2",
                    "on_base": {
                        "base1": "6",
                        "base2": "0",
                        "base3": "0"
                    },
                    "appearance_number": 2,
                    "strike_zone": [
                        3.329,
                        1.615,
                        0.75,
                        -0.75
                    ],
                    "full_result": "대타 서호철|서호철 : 중견수 플라이 아웃",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    -0.7218203864624638,
                                    2.284833535689379
                                ]
                            ],
                            "speed": "133",
                            "count": "0-1",
                            "pitch_result": "파울",
                            "event": null
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    -0.7313573529684394,
                                    2.1760150251423602
                                ]
                            ],
                            "speed": "133",
                            "count": "0-2",
                            "pitch_result": "스트라이크",
                            "event": null
                        },
                        {
                            "pitch_num": 3,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    1.4284755369128663,
                                    3.4859238390085614
                                ]
                            ],
                            "speed": "146",
                            "count": "1-2",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 4,
                            "pitch_type": "커브",
                            "pitch_coordinate": [
                                [
                                    1.7014443895057707,
                                    1.2493566991948706
                                ]
                            ],
                            "speed": "115",
                            "count": "2-2",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 5,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    0.17698295647015272,
                                    2.6440611088373487
                                ]
                            ],
                            "speed": "132",
                            "count": "2-2",
                            "pitch_result": "타격",
                            "event": null
                        }
                    ]
                },
                {
                    "inning": 9,
                    "half": "bot",
                    "pitcher": "61666",
                    "bat_order": 8,
                    "original_batter": null,
                    "actual_batter": "50901",
                    "out": "2",
                    "score": "9:2",
                    "on_base": {
                        "base1": "6",
                        "base2": "0",
                        "base3": "0"
                    },
                    "appearance_number": 1,
                    "strike_zone": [
                        3.323,
                        1.612,
                        0.75,
                        -0.75
                    ],
                    "full_result": "안인산 : 중견수 플라이 아웃",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    -0.3296011994438843,
                                    2.4564535892088823
                                ]
                            ],
                            "speed": "145",
                            "count": "0-0",
                            "pitch_result": "타격",
                            "event": null
                        }
                    ],
                    "main_result": "중견수 플라이 아웃"
                },
                {
                    "inning": 9,
                    "half": "bot",
                    "pitcher": "61666",
                    "bat_order": 9,
                    "original_batter": null,
                    "actual_batter": "64022",
                    "out": "3",
                    "score": "9:2",
                    "on_base": {
                        "base1": "6",
                        "base2": "0",
                        "base3": "0"
                    },
                    "appearance_number": 1,
                    "strike_zone": [
                        3.237,
                        1.57,
                        0.75,
                        -0.75
                    ],
                    "full_result": "안중열 : 삼진 아웃",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    -0.3214032311851819,
                                    1.7998801806888647
                                ]
                            ],
                            "speed": "133",
                            "count": "0-1",
                            "pitch_result": "스트라이크",
                            "event": null
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "커브",
                            "pitch_coordinate": [
                                [
                                    2.6253715818215024,
                                    -0.6299354659471188
                                ]
                            ],
                            "speed": "114",
                            "count": "1-1",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 3,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    1.1604821582069043,
                                    1.8609382262776664
                                ]
                            ],
                            "speed": "134",
                            "count": "2-1",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 4,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.8236813571019375,
                                    1.5596565704220733
                                ]
                            ],
                            "speed": "145",
                            "count": "2-2",
                            "pitch_result": "스트라이크",
                            "event": null
                        },
                        {
                            "pitch_num": 5,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    0.17136184027644394,
                                    1.550312804451011
                                ]
                            ],
                            "speed": "134",
                            "count": "2-3",
                            "pitch_result": "스트라이크",
                            "event": null
                        }
                    ],
                    "main_result": "삼진 아웃"
                }
            ]
        },
        "defense_positions": {
            "home_team": "NC",
            "away_team": "HH",
            "home": {
                "유격수": "김주원",
                "중견수": "",
                "2루수": "박민우",
                "1루수": "데이비슨",
                "우익수": "박건우",
                "지명타자": "안인산",
                "좌익수": "",
                "3루수": "김휘집",
                "포수": "안중열",
                "투수": "최성영"
            },
            "away": {
                "포수": "최재훈",
                "지명타자": "손아섭",
                "중견수": "리베라토",
                "좌익수": "문현빈",
                "3루수": "노시환",
                "1루수": "채은성",
                "우익수": "이진영",
                "2루수": "안치홍",
                "투수": "김범수",
                "11": "하주석",
                "유격수": "심우준"
            }
        }
    }
}""")

## test

In [35]:
inning_pack_to_pli_scores(raw_data)

[0.12,
 0.216,
 0.216,
 0.216,
 0.12,
 0.12,
 0.12,
 0.12,
 0.12,
 0.216,
 0.216,
 0.216,
 0.216,
 0.12,
 0.12]